In [1]:
import argparse
import os,sys
import logging
sys.path.append('object_detection')
sys.path.append('slim')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
from preprocessing import preprocessing_factory
from nets import nets_factory

from utils import visualization_utils as vis_util
from utils import label_map_util

import datetime
from flask import request, send_from_directory
from flask import Flask, request, redirect, url_for
import uuid
import math
import cv2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('checkpoint_dir', './faster_rcnn_nas_lowproposals_coco_2018_01_28', '')
tf.app.flags.DEFINE_string('slim_checkpoint_dir', './slim', '')
tf.app.flags.DEFINE_string('dataset_dir', './data_quiz_car', '')
tf.app.flags.DEFINE_string('output_dir', './output', '')
tf.app.flags.DEFINE_integer('num_top_predictions', 5,
                            """Display this many predictions.""")
tf.app.flags.DEFINE_string('label_dir', './slim', '')
tf.app.flags.DEFINE_integer('port', '5001', '')
tf.app.flags.DEFINE_boolean('debug', False, '')
tf.app.flags.DEFINE_string('upload_folder', '/tmp/', '')
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [3]:
NUM_CLASSES = 90
MAX_INFERENCE_NUM = 15
UPLOAD_FOLDER = FLAGS.upload_folder

In [4]:
app = Flask(__name__)
app._static_folder = FLAGS.upload_folder

In [5]:
network_fn = nets_factory.get_network_fn('inception_v3',
                                        num_classes=764,
                                        is_training=False)
image_size = network_fn.default_image_size
image_preprocessing_fn = preprocessing_factory.get_preprocessing(
            'inception_v3',
            is_training=False)

In [6]:
img_path = os.path.join(FLAGS.dataset_dir, 'car.jpg')  #image输入 
PATH_DETECTION_CKPT = os.path.join(FLAGS.checkpoint_dir, 'frozen_inference_graph.pb')
PATH_TO_LABELS = os.path.join(FLAGS.dataset_dir, 'mscoco_label_map.pbtxt')
PATH_CLASSIFY_CKPT = os.path.join(FLAGS.slim_checkpoint_dir, 'freezen_graph.pb')
LABEL = os.path.join(FLAGS.label_dir,'labels.txt')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
ALLOWED_EXTENSIONS = set(['jpg', 'JPG', 'jpeg', 'JPEG', 'png'])

In [8]:
# print(category_index)

In [9]:
graph = tf.Graph()
with graph.as_default():
    starttime = datetime.datetime.now()
    detection_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_DETECTION_CKPT,'rb') as f:
        detection_graph_def.ParseFromString(f.read())
        tf.import_graph_def(detection_graph_def, name='')
        detection_sess = tf.Session(graph=graph)
        endtime = datetime.datetime.now()
        print (endtime - starttime)
    #载入分类模型pb文件
    starttime = datetime.datetime.now()
    classify_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_CLASSIFY_CKPT, 'rb') as f:
        classify_graph_def.ParseFromString(f.read())
        tf.import_graph_def(classify_graph_def, name='')
        classify_sess = tf.Session(graph=graph)
        endtime = datetime.datetime.now()
        print (endtime - starttime)

0:00:05.141249
0:00:00.727054


In [10]:
def allowed_files(filename):
    return '.' in filename and \
        filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

In [11]:
def rename_filename(old_file_name):
    basename = os.path.basename(old_file_name)
    name, ext = os.path.splitext(basename)
    new_name = str(uuid.uuid1()) + ext
    return new_name

In [12]:
#定义将image图片转成numpyarray函数
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [13]:
#做一个中央裁切，用在分类模型上的函数
def preprocess_image(image, height, width,
                        central_fraction=0.875, scope=None):
  image = load_image_into_numpy_array(image)
  image = np.true_divide(image,255).astype('float32')

  H,W,_ = image.shape
  x = W/2
  y = H/2
  winW = W*central_fraction/2
  winH = H*central_fraction/2
  image = image[math.floor(y-winH):math.ceil(y + winH),math.floor(x-winW):math.ceil(x + winW)]

  image = cv2.resize(image, (height, width),interpolation=cv2.INTER_LINEAR)
  image = (image-0.5)*2
  return image

In [14]:
def inference_detect(img_path,graph,sess):
    image_tensor = graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = graph.get_tensor_by_name('detection_boxes:0')
    detection_classes = graph.get_tensor_by_name('detection_classes:0')
    image = Image.open(img_path)
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    starttime = datetime.datetime.now()
    print('before sess.run')
    (boxes, classes) = sess.run(
        [detection_boxes, detection_classes],
        feed_dict={image_tensor: image_np_expanded})
    print('after sess.run')
    endtime = datetime.datetime.now()
    print (endtime - starttime)
    
    boxes_squeeze = np.squeeze(boxes)
    classes_squeeze = np.squeeze(classes)
    
    boxnum = 0
    newboxes = []
    for i in range(len(classes_squeeze)):
        if classes_squeeze[i] == 3 or classes_squeeze[i] == 6 or classes_squeeze[i] == 8:
            newboxes.append(boxes_squeeze[i])
            boxnum += 1
            if boxnum >= MAX_INFERENCE_NUM:
                break
    print(classes_squeeze)
    print(boxnum)
    print(newboxes)
    return image, image_np, newboxes, boxnum

In [15]:
class NodeLookup(object):
  """Converts integer node ID's to human readable labels."""

  def __init__(self,
               label_path=None):
    if not label_path:
      tf.logging.fatal('please specify the label file.')
      return
    self.node_lookup = self.load(label_path)

  def load(self, label_path):
    """Loads a human readable English name for each softmax node.

    Args:
      label_lookup_path: string UID to integer node ID.
      uid_lookup_path: string UID to human-readable string.

    Returns:
      dict from integer node ID to human-readable string.
    """
    if not tf.gfile.Exists(label_path):
      tf.logging.fatal('File does not exist %s', label_lookup_path)

    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = tf.gfile.GFile(label_path).readlines()
    id_to_human = {}
    for line in proto_as_ascii_lines:
      if line.find(':') < 0:
        continue
      _id, human = line.rstrip('\n').split(':')
      id_to_human[int(_id)] = human

    return id_to_human

  def id_to_string(self, node_id):
    if node_id not in self.node_lookup:
      return ''
    return self.node_lookup[node_id]

In [16]:
def run_inference_on_image(imageArr, sess, boxnum):
    softmax_tensor = sess.graph.get_tensor_by_name('final_probs:0')
    predictions = sess.run(softmax_tensor,
                         {'input:0': imageArr})
    print("predictions:",predictions)
    node_lookup = NodeLookup(LABEL)

    carnames = []
    scores = []
    classids = []
    if boxnum == 1:
        prediction = np.array(predictions)
        for i in range(1):
            top_k = prediction.argsort()[-FLAGS.num_top_predictions:][::-1]
            print('top_k:')
            print(top_k)
            node_id = top_k[0]
            print('node_id')
            print(node_id)
        
            score = prediction[node_id]

            carname = node_lookup.id_to_string(node_id)
        
            scores.append(score) 
            classids.append(node_id)
            carnames.append(carname)
            print('id:[%d] name:[%s] (score = %.5f)' % (node_id, carname, score))
    else:
        print("predictions.shape")
        print(predictions.shape)
        for i in range(predictions.shape[0]):
            prediction = predictions[i]
            print("predictions[i]:",predictions[i])
            top_k = prediction.argsort()[-FLAGS.num_top_predictions:][::-1]
            print('top_k:')
            print(top_k)
            node_id = top_k[0]
            print('node_id')
            print(node_id)
        
            score = prediction[node_id]

            carname = node_lookup.id_to_string(node_id)
        
            scores.append(score) 
            classids.append(node_id)
            carnames.append(carname)
            print('id:[%d] name:[%s] (score = %.5f)' % (node_id, carname, score))
    print(carnames)
    print(scores)
    return carnames, scores, classids

In [17]:
#检测和分类结合部分
def inference(img_path,graph,sess1,sess2):
    starttimeall = datetime.datetime.now()
    imageArr = []
    with graph.as_default():
      image, image_np, newboxes, boxnum = inference_detect(img_path,graph,sess1)
      if len(newboxes) > 0:
          starttimett = datetime.datetime.now()
          for i in range(len(newboxes)):
              bbox = (image.size[0]*newboxes[i][1], image.size[1]*newboxes[i][0], image.size[0]*newboxes[i][3], image.size[1]*newboxes[i][2])
              bbox=tuple(bbox)
              try:
                  starttime = datetime.datetime.now()
                  newim=image.crop(bbox)      #裁剪图片
                  endtime = datetime.datetime.now()
                  print('finish image.crop(bbox)')
                  print (endtime - starttime)

                  starttime = datetime.datetime.now()
                  newim = preprocess_image(newim, image_size, image_size)                   
                  endtime = datetime.datetime.now()
                  
                  print('finish image_preprocessing_fn')
                  print (endtime - starttime)
                  
                  starttime = datetime.datetime.now()
                  imageArr.append(newim)      #裁剪后的图片添加到列表
                  endtime = datetime.datetime.now()
                  print('finish newim.eval(session=sess1)')
                  print (endtime - starttime)

              except SystemError:
                      print("crop Error")
          endtimett = datetime.datetime.now()
          print('total process img')
          print (endtimett - starttimett)

          print('befor run_inference_on_image')
          starttime = datetime.datetime.now()
          #导入分类模型as classes
          carnames, scores, classids = run_inference_on_image(imageArr, sess2, boxnum)
          endtime = datetime.datetime.now()
          print('after run_inference_on_image')
          print (endtime - starttime)
          
          vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              newboxes,
              classids,
              scores,
              category_index,
              use_normalized_coordinates=True,
              line_thickness=1,
              carnames = carnames)
          plt.imsave(os.path.join(UPLOAD_FOLDER, img_path), image_np)
          new_url = '/static/%s' % os.path.basename(img_path)
          print(new_url)
          image_tag = '<img src=%s></img><p>'
          new_tag = image_tag % new_url
          
          format_string = '<b>图片中有以下车型：</b><br/>'
          for score, carname in zip(scores, carnames):
              format_string += '%s (相似度:%.5f&#37;)<BR>' % (carname, score*100)
          
          ret_string = new_tag  + format_string + '<BR>' 
      else:
          new_url = '/static/%s' % os.path.basename(img_path)
          image_tag = '<img src=%s></img><p>'
          new_tag = image_tag % new_url
          
          ret_string = new_tag  + '<b>图片中没有发现任何汽车</b><br/><BR>' 
            
    endtimeall = datetime.datetime.now()
    print('all inference')
    print (endtimeall - starttimeall)
    ret_string = ret_string + '<b>预测耗时：%s</b><br/><BR>' % (endtimeall - starttimeall)
    return ret_string

In [18]:
inference_detect(img_path,graph,detection_sess)

before sess.run
after sess.run
0:00:54.636383
[ 8. 85.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]
1
[array([0.30204865, 0.13350688, 0.88516074, 0.8390426 ], dtype=float32)]


(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=543x326 at 0x193D286C6D8>,
 array([[[213, 218, 224],
         [213, 218, 224],
         [214, 219, 225],
         ...,
         [209, 212, 219],
         [209, 212, 219],
         [208, 211, 218]],
 
        [[213, 218, 224],
         [213, 218, 224],
         [214, 219, 225],
         ...,
         [209, 212, 219],
         [209, 212, 219],
         [209, 212, 219]],
 
        [[213, 218, 224],
         [213, 218, 224],
         [214, 219, 225],
         ...,
         [210, 213, 220],
         [209, 212, 219],
         [209, 212, 219]],
 
        ...,
 
        [[166, 170, 173],
         [167, 171, 174],
         [167, 171, 174],
         ...,
         [184, 185, 189],
         [185, 186, 190],
         [184, 185, 189]],
 
        [[171, 175, 178],
         [170, 174, 177],
         [169, 173, 176],
         ...,
         [183, 184, 188],
         [184, 185, 189],
         [183, 184, 188]],
 
        [[178, 182, 185],
         [1

In [19]:
@app.route("/", methods=['GET', 'POST'])
def root():
  result = """
    <!doctype html>
    <title>车辆检测项目</title>
    <h1>请上传一张需要检测汽车的照片吧！</h1>
    
    <form action="" method=post enctype=multipart/form-data>
      <p><input type=file name=file value='选择图片'>
      <br/><br/>
         <input type=submit value='上传预测'>
    </form>
    <p>%s</p>
    """ % "<br>"

  if request.method == 'POST':
    file = request.files['file']
    old_file_name = file.filename
    if file and allowed_files(old_file_name):
      filename = rename_filename(old_file_name)
      file_path = os.path.join(UPLOAD_FOLDER, filename)
      file.save(file_path)
      type_name = 'N/A'
      print('file saved to %s' % file_path)
    out_html = inference(file_path,graph,detection_sess,classify_sess)
    return result + out_html 
  return result

In [ ]:
print('listening on port %d' % FLAGS.port)
app.run(host='127.0.0.1', port=FLAGS.port, debug=FLAGS.debug, threaded=True)

listening on port 5001
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Oct/2019 11:48:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:48:02] "GET /favicon.ico HTTP/1.1" 404 -


file saved to /tmp/eba8441e-f547-11e9-883c-505bc2bd65aa.jpg
before sess.run


127.0.0.1 - - [23/Oct/2019 11:48:13] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:48:13] "GET /static/eba8441e-f547-11e9-883c-505bc2bd65aa.jpg HTTP/1.1" 200 -


after sess.run
0:00:07.278530
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
0
[]
all inference
0:00:07.439096
file saved to /tmp/f396a7ca-f547-11e9-bc24-505bc2bd65aa.jpg
before sess.run
after sess.run
0:00:07.047107
[ 8. 85.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]
1
[array([0.30204865, 0.13350688, 0.88516074, 0.8390426 ], dtype=float32)]
finish image.crop(bbox)
0:00:00.000996
finish image_preprocessing_fn
0:00:00.075833
finish newim.eval(session=sess1)
0:00:00
total process img
0:00:00.076829
befor run_inference_on_image


127.0.0.1 - - [23/Oct/2019 11:48:32] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:48:32] "GET /static/f396a7ca-f547-11e9-bc24-505bc2bd65aa.jpg HTTP/1.1" 200 -


predictions: [2.38515750e-05 3.30917874e-06 1.58262374e-05 1.07138701e-01
 1.75379810e-03 8.88793647e-01 3.84219402e-06 2.52859718e-06
 1.02622153e-06 3.30380863e-06 3.78849080e-07 5.07905816e-06
 3.19439692e-07 5.26638416e-07 1.75910475e-06 7.92752985e-07
 1.30511557e-06 6.11353641e-07 6.62080708e-07 5.60916533e-07
 1.76967012e-06 3.58146025e-07 1.10750727e-06 1.42492851e-07
 3.14794818e-07 1.74240677e-06 3.25704661e-07 1.88743911e-07
 7.06454784e-07 2.80292085e-07 7.96147276e-07 4.53364635e-07
 7.28079783e-07 8.85638315e-07 2.17968193e-07 4.72508020e-07
 4.75656094e-07 2.52497534e-06 4.26113587e-07 1.72808313e-07
 5.10131358e-05 5.58466309e-06 8.94613606e-07 2.15068258e-06
 3.84940989e-07 5.42828445e-07 3.55194913e-07 3.13041085e-07
 1.15852663e-06 1.48815377e-06 3.39935440e-07 1.94315106e-07
 2.55937607e-07 6.78126696e-07 1.17058597e-07 2.09568014e-07
 5.33851846e-07 1.18266610e-06 4.08009873e-06 7.14332486e-07
 5.38119480e-07 1.82819846e-07 2.91556915e-07 1.62526214e-06
 9.29672694

file saved to /tmp/fe3538ac-f547-11e9-a961-505bc2bd65aa.jpg
before sess.run
after sess.run
0:00:07.209680
[3. 3. 3. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
4
[array([0.3908984 , 0.2867518 , 0.75035197, 0.5547124 ], dtype=float32), array([0.34649363, 0.6752062 , 0.55369   , 0.8639686 ], dtype=float32), array([0.37525153, 0.5088359 , 0.6094829 , 0.7112462 ], dtype=float32), array([0.47313726, 0.16757694, 0.6849819 , 0.28531066], dtype=float32)]
finish image.crop(bbox)
0:00:00
finish image_preprocessing_fn
0:00:00.073803
finish newim.eval(session=sess1)
0:00:00
finish image.crop(bbox)
0:00:00.000997
finish image_preprocessing_fn
0:00:00.027926
finish newim.eval(session=sess1)
0:00:00
finish image.crop(bbox)
0:00:00
finish image_preprocessing_fn
0:00:00.034906
finish newim.eval(session=sess1)
0:00:00
finish image.crop(bbox)
0:00:00
finish image_preprocessing_fn
0:00:00.033910
finish newim.eval(session=sess1)
0:00:00
total process img
0:00:00.172536
befor run_inference_on_image


127.0.0.1 - - [23/Oct/2019 11:48:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:48:45] "GET /static/fe3538ac-f547-11e9-a961-505bc2bd65aa.jpg HTTP/1.1" 200 -


predictions: [[1.3592425e-05 2.7023671e-05 6.8163150e-05 ... 4.7552903e-06
  6.8568256e-06 5.8454602e-06]
 [5.8063182e-05 7.4643751e-05 7.3125563e-04 ... 3.3738619e-05
  3.6336642e-05 1.1095618e-04]
 [1.5408336e-06 5.6440367e-06 2.3544635e-06 ... 1.4506052e-06
  5.4824812e-07 6.0960707e-07]
 [8.5317683e-05 1.3430762e-03 1.1185286e-02 ... 1.4081681e-05
  1.6511236e-05 1.7136481e-05]]
predictions.shape
(4, 764)
predictions[i]: [1.35924247e-05 2.70236706e-05 6.81631500e-05 5.03184310e-05
 3.09397401e-05 5.25445139e-05 1.09085944e-04 4.32359120e-05
 1.91990915e-03 1.66773490e-04 1.54966416e-04 1.17379306e-04
 6.02745313e-05 1.09965535e-04 2.03579584e-05 1.12844376e-04
 6.42232480e-05 1.80251973e-05 1.50266569e-05 9.68326349e-05
 4.67040036e-06 6.57282908e-06 3.06947641e-06 6.11142514e-06
 1.01819842e-05 4.09679778e-05 5.32885315e-06 9.22019990e-06
 5.29586978e-05 5.62955283e-05 9.45263819e-05 6.74855710e-06
 5.55054976e-06 1.28136699e-05 4.25310873e-06 6.24034783e-06
 1.36323042e-05 4.1934